## **Stage-1**

Creating a knowledge base system which will act as brain to my model

In [8]:
import os 
import re
import pickle
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

## Configuration 
DEFAULT_MODEL_NAME="sentence-transformers/all-MiniLM-L6-v2"

def clean_text(text:str) -> str:
    """This function is used to clean the entire text [description]"""
    if text is None:
        return ""
    text = str(text)
    text = re.sub(r'\s+',' ',text).strip()
    return text

def safe_to_str(x):
    """This is a helper function"""
    if pd.isna(x):
        return ""
    return str(x).strip()

def build_system_kb_store_all_columns(
        excel_path: str,
        save_dir: str = "system_kb_store",
        model_name: str = DEFAULT_MODEL_NAME,
        batch_size: int = 64,
        embed_column: str = 'Description of Contract'
):
    """This function will create embedding on my description for my knowledge base"""
    os.makedirs(save_dir,exist_ok=True)
    print(f"\n Loading Excel Knowledge Base: {excel_path}")
    df = pd.read_excel(excel_path)
    print(f"Loaded rows={len(df)} col={len(df.columns)}")

    if embed_column not in df.columns:
        raise ValueError(f"Embed column '{embed_column}' not found in Excel!")
    
    df = df.fillna('')
    kb_texts=[]
    kb_meta=[]
    
    for idx, row in df.iterrows():
        desc=clean_text(row[embed_column])
        if not desc or len(desc) < 20:
            continue
        meta = {'row_id':int(idx)}
        for col in df.columns:
            meta[col]=safe_to_str(row[col])
        meta[embed_column]=desc
        kb_texts.append(desc)
        kb_meta.append(meta)
    print(f"KB rows kept after cleaning: {len(kb_texts)}")
    if len(kb_texts) == 0:
        print("ERROR:No text rows remained after cleaning. Check your 'clean_text' logic or input data.")
        return None, None
    print(f"Loading embedding model: {model_name}")
    embedder=SentenceTransformer(model_name)
    print("Creating Embeddings...")
    embeddings=[]
    for i in range(0,len(kb_texts),batch_size):
        batch=kb_texts[i:i + batch_size]
        batch_emb=embedder.encode(batch,show_progress_bar=True,normalize_embeddings=True)
        embeddings.append(batch_emb)
    embeddings=np.vstack(embeddings).astype('float32')
    dim=embeddings.shape[1]
    print(f"Embedding Shape:{embeddings.shape}")
    index=faiss.IndexFlatIP(dim)
    index.add(embeddings)
    index_path=os.path.join(save_dir,"system_kb.faiss")
    meta_path=os.path.join(save_dir,'system_kb_meta.pkl')
    faiss.write_index(index,index_path)
    with open(meta_path,'wb') as f:
        pickle.dump(kb_meta,f)
    print("System KB Created Successfully!")
    print(f"Index saved: {index_path}")
    print(f"Meta Saved: {meta_path}")
    return index_path, meta_path

if __name__=="__main__":
    EXCEL_PATH=r"C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\data\sample_data.xlsx"
    build_system_kb_store_all_columns(
        excel_path=EXCEL_PATH,
        save_dir='system_kb_store',
        model_name=DEFAULT_MODEL_NAME,
        batch_size=64,
        embed_column="Description of Contract"
    )




 Loading Excel Knowledge Base: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\data\sample_data.xlsx
Loaded rows=2068 col=29
KB rows kept after cleaning: 600
Loading embedding model: sentence-transformers/all-MiniLM-L6-v2


'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: b94d340d-9491-4534-9001-a3ba8a140304)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


Creating Embeddings...


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.01it/s]

Embedding Shape:(600, 384)
System KB Created Successfully!
Index saved: system_kb_store\system_kb.faiss
Meta Saved: system_kb_store\system_kb_meta.pkl


In [11]:
import os 
import pickle
import faiss
from sentence_transformers import SentenceTransformer

DEFAULT_MODEL_NAME='sentence-transformers/all-MiniLM-L6-v2'

class SystemKBRetriever:
    def __init__(self, kb_dir='system_kb_store', model_name=DEFAULT_MODEL_NAME):
        index_path = os.path.join(kb_dir, 'system_kb.faiss')
        meta_path = os.path.join(kb_dir, 'system_kb_meta.pkl')

        if not os.path.exists(index_path) or not os.path.exists(meta_path):
            raise FileNotFoundError("KB files are missing. Build KB first.")
        
        print(f"Loading FAISS index: {index_path}")
        self.index = faiss.read_index(index_path)
        
        print(f"Loading metadata: {meta_path}")
        with open(meta_path, 'rb') as f:
            self.meta = pickle.load(f)
            
        print(f"Loaded KB rows: {len(self.meta)}")
        print(f"Loading embedder: {model_name}")
        self.embedder = SentenceTransformer(model_name)
    
    def retrieve(self, query_text: str, top_k: int = 5):
        # FIX 1: Do not use .split(). We want to embed the whole sentence, not a list of words.
        query_text = str(query_text).strip()
        
        if not query_text:
            return []
        
        # Encode returns shape (1, 384) because we pass a list with 1 string
        q_emb = self.embedder.encode([query_text], normalize_embeddings=True).astype('float32')
        
        scores, idxs = self.index.search(q_emb, top_k)
        results = []
        
        # FIX 2: Corrected typo 'socre' -> 'score'
        for score, idx in zip(scores[0], idxs[0]):
            if idx < 0:
                continue
            results.append({
                'score': float(score),  # FIX 3: Use individual 'score', not the array 'scores'
                "meta": self.meta[idx]
            })
        return results
    
if __name__=="__main__":
    # Ensure the directory exists and contains files created by the builder script
    if os.path.exists('system_kb_store'):
        r = SystemKBRetriever(kb_dir='system_kb_store')
        q = 'Dell Marketing L.P., Round Rock, Texas, is awarded a single-award, firm-fixed-price blanket purchase agreement'
        
        hits = r.retrieve(q, top_k=3)

        for h in hits:
            print("\nScore:", h['score'])
            print("Supplier:", h['meta'].get('Supplier Name'))
            print("Market:", h['meta'].get("Market Segment"))
            print("System:", h['meta'].get('System Name (Specific)'))
    else:
        print("Error: 'system_kb_store' directory not found. Please run the builder script first.")

'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: a7b0a49c-5ed7-4137-a283-22884e9d1ce4)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


Loading FAISS index: system_kb_store\system_kb.faiss
Loading metadata: system_kb_store\system_kb_meta.pkl
Loaded KB rows: 600
Loading embedder: sentence-transformers/all-MiniLM-L6-v2

Score: 0.6228203773498535
Supplier: Dell Inc
Market: Unknown
System: Department of Defense Enterprise Software Initiative (DOD ESI)

Score: 0.6228203773498535
Supplier: Dell Inc
Market: Unknown
System: Department of Defense Enterprise Software Initiative (DOD ESI)

Score: 0.6024371385574341
Supplier: 
Market: Unknown
System: Department of Defense Enterprise Software Initiative (DOD ESI)


In [12]:
import os
import json
import difflib
import pandas as pd
import datetime
from dateutil import parser
from dateutil.relativedelta import relativedelta
import getpass
from typing import Annotated, TypedDict, List
import re
import pickle
import faiss

# LangChain / LangGraph Imports
from langchain_core.messages import AnyMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from openai import OpenAI

# ==============================================================================
# RAG RETRIEVER (Single File Implementation)
# ==============================================================================

class SystemKBRetriever:
    """
    Loads FAISS index + metadata created from your KB excel.
    Uses ONLY the contract paragraph to retrieve similar examples.
    """
    def __init__(self, kb_dir: str):
        index_path = os.path.join(kb_dir, "system_kb.faiss")
        meta_path = os.path.join(kb_dir, "system_kb_meta.pkl")

        if not os.path.exists(index_path) or not os.path.exists(meta_path):
            raise FileNotFoundError(
                f"❌ KB files not found in: {kb_dir}\n"
                f"Expected:\n- {index_path}\n- {meta_path}\n\n"
                f"Build the KB first using your KB builder script."
            )

        print(f"Loading FAISS Index: {index_path}")
        self.index = faiss.read_index(index_path)

        print(f"Loading KB Metadata: {meta_path}")
        with open(meta_path, "rb") as f:
            self.meta = pickle.load(f)

        print(f"KB Loaded: {len(self.meta)} rows")

    def retrieve(self, query_text: str, top_k: int = 3):
        """
        Returns:
        [
          {"score": float, "meta": {...all 29 cols...}},
          ...
        ]
        """
        import numpy as np
        from sentence_transformers import SentenceTransformer

        query_text = str(query_text).strip()
        if not query_text:
            return []

        # Use embedding model only when needed (lazy load)
        if not hasattr(self, "embedder"):
            self.embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

        q_emb = self.embedder.encode([query_text], normalize_embeddings=True).astype("float32")
        scores, idxs = self.index.search(q_emb, top_k)

        results = []
        for score, idx in zip(scores[0], idxs[0]):
            if idx < 0:
                continue
            results.append({
                "score": float(score),
                "meta": self.meta[idx]
            })

        return results


# ==============================================================================
# 1. CONFIGURATION & FILE PATHS
# ==============================================================================

# UPDATE PATHS HERE
TAXONOMY_PATH = r'C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\notebook\taxonomy.json'
SUPPLIERS_PATH = r'C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\notebook\suppliers.json'
INPUT_EXCEL_PATH = r"C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\data\source_file.xlsx"
OUTPUT_EXCEL_PATH = "Processed_Defense_Data.xlsx"

# RAG KB Directory (must contain system_kb.faiss + system_kb_meta.pkl)
RAG_KB_DIR = r"C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\notebook\system_kb_store"

# Setup API Key
if "LLMFOUNDRY_TOKEN" not in os.environ:
    os.environ["LLMFOUNDRY_TOKEN"] = getpass.getpass("Enter the LLM Foundry API Key: ")

# Shared Client
client = OpenAI(
    api_key=f'{os.environ.get("LLMFOUNDRY_TOKEN")}:my-test-project',
    base_url="https://llmfoundry.straive.com/openai/v1/",
)

# Load retriever once globally
retriever = SystemKBRetriever(kb_dir=RAG_KB_DIR)

# --- FILE LOADING HELPERS ---
def load_json_file(filename, default_value):
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            print(f"Loaded {filename}")
            return json.load(f)
    except Exception as e:
        print(f"Warning: Could not load {filename} ({e}). Using default.")
        return default_value

# 1. Load Taxonomy
raw_taxonomy = load_json_file(TAXONOMY_PATH, {})
TAXONOMY_STR = json.dumps(raw_taxonomy, separators=(',', ':'))

# 2. Load Suppliers
SUPPLIER_LIST = load_json_file(SUPPLIERS_PATH, [
    "Dell Inc", "Boeing", "Lockheed Martin", "Raytheon Technologies",
    "Northrop Grumman", "L3Harris", "BAE Systems", "General Dynamics"
])

# 3. System Rules
RULE_BOOK = {
    "defensive_countermeasures": {
        "triggers": ["flare", "chaff", "countermeasure", "decoy", "mju-", "ale-"],
        "guidance": "Market Segment: 'C4ISR Systems', System Type (General): 'Defensive Systems', Specific: 'Defensive Aid Suite'"
    },
    "radars_and_sensors": {
        "triggers": ["radar", "sonar", "sensor", "an/apy", "an/tpy"],
        "guidance": "Market Segment: 'C4ISR Systems', System Type (General): 'Sensors'"
    },
    "ammunition": {
        "triggers": ["cartridge", "round", "projectile", " 5.56", " 7.62", "ammo"],
        "guidance": "Market Segment: 'Weapon Systems', System Type (General): 'Ammunition'"
    }
}

# 4. Geography Mapping
GEOGRAPHY_MAPPING = {
    "North America": ["USA", "United States", "US", "Canada", "America"],
    "Europe": ["UK", "United Kingdom", "Ukraine", "Germany", "France", "Italy", "Spain", "Poland", "Netherlands", "Norway", "Sweden", "Finland", "Denmark", "Belgium"],
    "Asia-Pacific": ["Australia", "Japan", "South Korea", "Taiwan", "India", "Singapore", "New Zealand"],
    "Middle East and North Africa": ["Israel", "Saudi Arabia", "UAE", "Egypt", "Qatar", "Kuwait", "Iraq"],
    "International Organisations": ["NATO", "EU", "IFU", "UN", "NSPA"]
}

# ==============================================================================
# 2. HELPER FUNCTIONS
# ==============================================================================

def get_best_supplier_match(extracted_name):
    """Fuzzy matches the extracted name against the loaded SUPPLIER_LIST."""
    if not extracted_name or extracted_name.lower() in ["unknown", "n/a"]:
        return "Unknown"

    clean_name = extracted_name.strip()

    supplier_map = {s.lower(): s for s in SUPPLIER_LIST}
    if clean_name.lower() in supplier_map:
        return supplier_map[clean_name.lower()]

    matches = difflib.get_close_matches(clean_name, SUPPLIER_LIST, n=1, cutoff=0.6)
    return matches[0] if matches else clean_name


def calculate_mro_months(start_date_str, end_date_text, program_type):
    """Calculates duration only if Program Type is MRO/Support."""
    if program_type != "MRO/Support":
        return "Not Applicable"

    try:
        if not start_date_str or not end_date_text or str(end_date_text).lower() in ["unknown", "n/a"]:
            return "Not Applicable"

        start = pd.to_datetime(start_date_str, dayfirst=True)
        end = parser.parse(end_date_text, fuzzy=True)

        diff = relativedelta(end, start)
        total_months = (diff.years * 12) + diff.months
        return str(max(0, int(total_months)))
    except:
        return "Not Applicable"


def get_region_for_country(country_name):
    """Robust lookup handling casing/whitespace."""
    if not country_name or country_name.lower() in ["unknown", "n/a", "not applicable"]:
        return "Unknown"

    clean = country_name.strip().lower()

    if clean in ["us", "usa", "united states", "united states of america"]:
        return "North America"
    if clean in ["uk", "britain", "great britain"]:
        return "Europe"

    for region, countries in GEOGRAPHY_MAPPING.items():
        if any(c.lower() == clean for c in countries):
            return region
    return "Unknown"


# Regex Designator Extractors (for System Name + Piloting)
DESIGNATOR_PATTERNS = [
    r"\bDDG[-\s]?\d+\b", r"\bCVN[-\s]?\d+\b", r"\bSSN[-\s]?\d+\b",
    r"\bLCS[-\s]?\d+\b", r"\bLPD[-\s]?\d+\b", r"\bLHA[-\s]?\d+\b", r"\bLHD[-\s]?\d+\b",
    r"\bF-\d+\b", r"\bB-\d+\b", r"\bC-\d+\b", r"\bA-\d+\b",
    r"\bMQ-\d+\b", r"\bRQ-\d+\b",
    r"\bAN\/[A-Z0-9\-]+\b",
    r"\b(AIM|AGM|SM|RIM|MIM)-\d+\b",
]

def extract_designators(text: str):
    text = str(text)
    found = []
    for pat in DESIGNATOR_PATTERNS:
        found.extend(re.findall(pat, text, flags=re.IGNORECASE))
    cleaned = []
    for f in found:
        cleaned.append(f.upper().replace(" ", "").replace("--", "-"))
    seen = set()
    final = []
    for x in cleaned:
        if x not in seen:
            final.append(x)
            seen.add(x)
    return final

def detect_piloting_rule_based(text: str, designators: List[str]) -> str:
    t = str(text).lower()

    if any(d.startswith(("MQ-", "RQ-")) for d in designators):
        return "Uncrewed"
    if any(k in t for k in ["unmanned", "uav", "drone", "autonomous"]):
        return "Uncrewed"

    if any(d.startswith(("DDG", "CVN", "SSN", "LCS", "LPD", "LHA", "LHD")) for d in designators):
        return "Crewed"
    if "uss " in t:
        return "Crewed"

    return "Not Applicable"

# ==============================================================================
# 3. TOOL DEFINITIONS (AGENTS)
# ==============================================================================

# --- AGENT 1: SOURCING ---
class SourcingInput(BaseModel):
    paragraph: str = Field(description="Contract text.")
    url: str = Field(description="Source URL.")
    date: str = Field(description="Contract Date.")

@tool("sourcing_extractor")
def sourcing_extractor(paragraph: str, url: str, date: str):
    """Stage 1: Prepares Metadata."""
    reported_date = datetime.datetime.now().strftime("%Y-%m-%d")
    notes = "Standard extraction."
    if "modification" in paragraph.lower():
        notes = "Contract Modification."
    if "split" in paragraph.lower():
        notes = "Split award detected."

    return {
        "Description of Contract": paragraph,
        "Additional Notes (Internal Only)": notes,
        "Source Link(s)": url,
        "Contract Date": date,
        "Reported Date (By SGA)": reported_date
    }


# --- AGENT 2: GEOGRAPHY ---
class GeographyInput(BaseModel):
    paragraph: str = Field(description="Contract text.")

@tool("geography_extractor")
def geography_extractor(paragraph: str):
    """Stage 2: Geography Logic."""
    sys_prompt = """
Extract: Customer Country, Supplier Country, Customer Operator.
Logic: If 'Navy awarded...', Operator is Navy.
Return JSON:
{
  "Customer Country": "...",
  "Customer Operator": "...",
  "Supplier Country": "..."
}
"""
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": paragraph}
            ],
            temperature=0,
            response_format={"type": "json_object"}
        )
        raw = json.loads(completion.choices[0].message.content)
    except Exception:
        raw = {}

    cust = raw.get("Customer Country", "Unknown")
    supp = raw.get("Supplier Country", "Unknown")

    domestic = "Indigenous" if cust.lower() == supp.lower() else "Imported"
    if "united states" in cust.lower() and "usa" in supp.lower():
        domestic = "Indigenous"

    return {
        "Customer Region": get_region_for_country(cust),
        "Customer Country": cust,
        "Customer Operator": raw.get("Customer Operator", "Unknown"),
        "Supplier Region": get_region_for_country(supp),
        "Supplier Country": supp,
        "Domestic Content": domestic
    }


# --- AGENT 3: SYSTEM (UPGRADED WITH RAG + Evidence + Reason) ---
class SystemInput(BaseModel):
    paragraph: str = Field(description="Contract text.")

@tool("system_classifier")
def system_classifier(paragraph: str):
    """Stage 3: System classification using RAG + Rule Book + Evidence & Reason."""
    paragraph = str(paragraph).strip()
    if not paragraph:
        return {}

    lower_text = paragraph.lower()

    # RULE BOOK triggers
    hints = [
        f"RULE: {v['guidance']}"
        for _, v in RULE_BOOK.items()
        if any(t in lower_text for t in v["triggers"])
    ]
    hint_str = "\n".join(hints) if hints else "No special override rules triggered."

    # Local extraction
    designators = extract_designators(paragraph)

    # Rule-based piloting
    piloting_rule = detect_piloting_rule_based(paragraph, designators)

    # RAG Retrieval
    rag_hits = retriever.retrieve(paragraph, top_k=3)

    rag_examples = []
    for hit in rag_hits:
        meta = hit["meta"]
        rag_examples.append({
            "score": round(hit["score"], 4),
            "Market Segment": meta.get("Market Segment", ""),
            "System Type (General)": meta.get("System Type (General)", ""),
            "System Type (Specific)": meta.get("System Type (Specific)", ""),
            "System Name (General)": meta.get("System Name (General)", ""),
            "System Name (Specific)": meta.get("System Name (Specific)", ""),
            "System Piloting": meta.get("System Piloting", ""),
            "Supplier Name": meta.get("Supplier Name", ""),
            "Customer Operator": meta.get("Customer Operator", ""),
            "Snippet": meta.get("Description of Contract", "")[:220] + "..."
        })

    sys_prompt = f"""
You are a Senior Defense System Classification Analyst.

Use these inputs:
1) RAG Similar Examples (top 3)
2) Rule Book Overrides
3) Extracted Designators

RULE BOOK OVERRIDES:
{hint_str}

OUTPUT RULES:
- Return ONLY a FLAT JSON object.
- Every value must be a STRING.
- Do NOT return nested JSON or lists.
- Evidence must be copied EXACTLY from paragraph.
- If evidence not present, use "Not Found".

Return JSON exactly:
{{
  "Market Segment": "",
  "Market Segment Evidence": "",
  "Market Segment Reason": "",

  "System Type (General)": "",
  "System Type (General) Evidence": "",
  "System Type (General) Reason": "",

  "System Type (Specific)": "",
  "System Type (Specific) Evidence": "",
  "System Type (Specific) Reason": "",

  "System Name (General)": "",
  "System Name (General) Evidence": "",
  "System Name (General) Reason": "",

  "System Name (Specific)": "",
  "System Name (Specific) Evidence": "",
  "System Name (Specific) Reason": "",

  "System Piloting": "",
  "System Piloting Evidence": "",
  "System Piloting Reason": "",

  "Confidence": "High/Medium/Low"
}}
"""

    user_prompt = f"""
INPUT PARAGRAPH:
{paragraph}

DESIGNATORS (regex extracted):
{designators if designators else "None"}

RULE-BASED PILOTING:
{piloting_rule}

RAG SIMILAR EXAMPLES:
{json.dumps(rag_examples, indent=2)}
"""

    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            response_format={"type": "json_object"}
        )
        result = json.loads(completion.choices[0].message.content)

        # Hard override piloting (best accuracy)
        result["System Piloting"] = piloting_rule
        if not result.get("System Piloting Reason"):
            result["System Piloting Reason"] = "Piloting derived using deterministic rules."
        if not result.get("System Piloting Evidence"):
            result["System Piloting Evidence"] = "Not Found"

        # Ensure flat
        for k, v in result.items():
            if isinstance(v, (dict, list)):
                result[k] = str(v)

        return result

    except Exception as e:
        return {
            "Market Segment": "",
            "System Type (General)": "",
            "System Type (Specific)": "",
            "System Name (General)": "",
            "System Name (Specific)": "",
            "System Piloting": piloting_rule,
            "Market Segment Evidence": "Not Found",
            "System Type (General) Evidence": "Not Found",
            "System Type (Specific) Evidence": "Not Found",
            "System Name (General) Evidence": "Not Found",
            "System Name (Specific) Evidence": "Not Found",
            "System Piloting Evidence": "Not Found",
            "Market Segment Reason": "",
            "System Type (General) Reason": "",
            "System Type (Specific) Reason": "",
            "System Name (General) Reason": "",
            "System Name (Specific) Reason": "",
            "System Piloting Reason": "Piloting derived using deterministic rules.",
            "Confidence": "Low",
            "Error": str(e)
        }


# --- AGENT 4: CONTRACT ---
class ContractInfoInput(BaseModel):
    paragraph: str = Field(description="Contract text.")
    contract_date: str = Field(description="Signed date.")

@tool("contract_extractor")
def contract_extractor(paragraph: str, contract_date: str):
    """Stage 4: Extracts Financials, Program Type, and Dates based on strict SOP."""

    system_instruction = """
You are a Defense Contract Financial Analyst. Extract data strictly following these SOP rules:

1) Supplier Name: Extract the exact company name text found in paragraph.
2) Program Type: Procurement / Training / MRO/Support / RDT&E / Upgrade / Other Service
3) Value Certainty:
   - Confirmed: fixed price/obligated stated
   - Estimated: ceiling/potential/IDIQ/multi-award
4) Quantity: number or Not Applicable
5) G2G/B2G:
   - G2G only if Foreign Military Sales (FMS)
   - else B2G
6) Completion Date: only needed for MRO duration calc

Return JSON only.
"""

    user_prompt = f"""
Analyze contract:
"{paragraph}"

Signed Date: {contract_date}

Return JSON:
{{
  "raw_supplier_name": "",
  "program_type": "",
  "value_million_raw": "",
  "currency_code": "",
  "value_certainty": "",
  "quantity": "",
  "completion_date_text": "",
  "g2g_b2g": "",
  "value_note": ""
}}
"""

    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_instruction},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            response_format={"type": "json_object"}
        )
        raw = json.loads(completion.choices[0].message.content)
    except Exception as e:
        return {"Error": str(e)}

    final_supplier = get_best_supplier_match(raw.get("raw_supplier_name"))
    prog_type = raw.get("program_type", "Unknown")
    mro_months = calculate_mro_months(contract_date, raw.get("completion_date_text"), prog_type)

    try:
        val_str = str(raw.get("value_million_raw", "0")).replace(",", "").replace("$", "")
        val_float = float(val_str)
        val_formatted = "{:.3f}".format(val_float)
    except:
        val_formatted = "0.000"

    try:
        dt = pd.to_datetime(contract_date)
        sign_month = dt.strftime("%B")
        sign_year = str(dt.year)
    except:
        sign_month, sign_year = "Unknown", "Unknown"

    val_note = raw.get("value_note", "Not Applicable")
    if "split" in paragraph.lower() and val_note == "Not Applicable":
        val_note = "Split contract; value distribution unclear."

    return {
        "Supplier Name": final_supplier,
        "Program Type": prog_type,
        "Expected MRO Contract Duration (Months)": mro_months,
        "Quantity": raw.get("quantity", "Not Applicable"),
        "Value Certainty": raw.get("value_certainty", "Confirmed"),
        "Value (Million)": val_formatted,
        "Currency": raw.get("currency_code", "USD$"),
        "Value (USD$ Million)": val_formatted,
        "Value Note (If Any)": val_note,
        "G2G/B2G": raw.get("g2g_b2g", "B2G"),
        "Signing Month": sign_month,
        "Signing Year": sign_year
    }


# ==============================================================================
# 4. LANGGRAPH PIPELINE
# ==============================================================================

class AgentState(TypedDict):
    input_text: str
    input_date: str
    input_url: str
    final_data: dict
    messages: Annotated[List[AnyMessage], add_messages]

def stage_1_sourcing(state: AgentState):
    res = sourcing_extractor.invoke({
        "paragraph": state["input_text"],
        "url": state["input_url"],
        "date": state["input_date"]
    })
    new_data = state.get("final_data", {}).copy()
    new_data.update(res)
    return {"final_data": new_data}

def stage_2_geography(state: AgentState):
    res = geography_extractor.invoke({"paragraph": state["input_text"]})
    new_data = state.get("final_data", {}).copy()
    new_data.update(res)
    return {"final_data": new_data}

def stage_3_system(state: AgentState):
    res = system_classifier.invoke({"paragraph": state["input_text"]})
    new_data = state.get("final_data", {}).copy()
    new_data.update(res)
    return {"final_data": new_data}

def stage_4_contract(state: AgentState):
    res = contract_extractor.invoke({
        "paragraph": state["input_text"],
        "contract_date": state["input_date"]
    })
    new_data = state.get("final_data", {}).copy()
    new_data.update(res)
    return {"final_data": new_data}

workflow = StateGraph(AgentState)
workflow.add_node("Stage1", stage_1_sourcing)
workflow.add_node("Stage2", stage_2_geography)
workflow.add_node("Stage3", stage_3_system)
workflow.add_node("Stage4", stage_4_contract)

workflow.add_edge(START, "Stage1")
workflow.add_edge("Stage1", "Stage2")
workflow.add_edge("Stage2", "Stage3")
workflow.add_edge("Stage3", "Stage4")
workflow.add_edge("Stage4", END)

app = workflow.compile()

# ==============================================================================
# 5. EXECUTION & FORMATTING
# ==============================================================================

if __name__ == "__main__":

    print(f"Loading Input File: {INPUT_EXCEL_PATH}...")

    try:
        df_input = pd.read_excel(INPUT_EXCEL_PATH)

        required_cols = ["Source URL", "Contract Date", "Contract Description"]
        if not all(col in df_input.columns for col in required_cols):
            raise ValueError(f"Excel file must contain columns: {required_cols}")

        print(f"🚀 Processing {len(df_input)} rows...")
        results = []

        for index, row in df_input.iterrows():
            print(f"   -> Row {index + 1}...")

            desc = str(row["Contract Description"]) if pd.notna(row["Contract Description"]) else ""
            c_date = str(row["Contract Date"]) if pd.notna(row["Contract Date"]) else str(datetime.date.today())
            c_url = str(row["Source URL"]) if pd.notna(row["Source URL"]) else ""

            initial_state = {
                "input_text": desc,
                "input_date": c_date,
                "input_url": c_url,
                "final_data": {},
                "messages": []
            }

            output_state = app.invoke(initial_state)
            results.append(output_state["final_data"])

        df_final = pd.DataFrame(results)

        FINAL_COLUMNS = [
            "Customer Region", "Customer Country", "Customer Operator",
            "Supplier Region", "Supplier Country", "Domestic Content",

            "Market Segment", "Market Segment Evidence", "Market Segment Reason",
            "System Type (General)", "System Type (General) Evidence", "System Type (General) Reason",
            "System Type (Specific)", "System Type (Specific) Evidence", "System Type (Specific) Reason",
            "System Name (General)", "System Name (General) Evidence", "System Name (General) Reason",
            "System Name (Specific)", "System Name (Specific) Evidence", "System Name (Specific) Reason",
            "System Piloting", "System Piloting Evidence", "System Piloting Reason",
            "Confidence",

            "Supplier Name", "Program Type", "Expected MRO Contract Duration (Months)",
            "Quantity", "Value Certainty", "Value (Million)", "Currency",
            "Value (USD$ Million)", "Value Note (If Any)", "G2G/B2G",
            "Signing Month", "Signing Year",

            "Description of Contract",
            "Additional Notes (Internal Only)",
            "Source Link(s)",
            "Contract Date",
            "Reported Date (By SGA)"
        ]

        df_final = df_final.reindex(columns=FINAL_COLUMNS, fill_value="")

        df_final.to_excel(OUTPUT_EXCEL_PATH, index=False)

        print("\n Processing Complete!")
        print(f"File saved to: {OUTPUT_EXCEL_PATH}")
        print(df_final.head().to_string())

    except Exception as e:
        print(f"\n Error: {e}")


Loading FAISS Index: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\notebook\system_kb_store\system_kb.faiss
Loading KB Metadata: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\notebook\system_kb_store\system_kb_meta.pkl
KB Loaded: 600 rows
Loaded C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\notebook\taxonomy.json
Loaded C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\notebook\suppliers.json
Loading Input File: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\data\source_file.xlsx...
🚀 Processing 1 rows...
   -> Row 1...


C:\Users\mukeshkr\AppData\Local\Temp\ipykernel_19764\1953562160.py:184: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  start = pd.to_datetime(start_date_str, dayfirst=True)



 Processing Complete!
File saved to: Processed_Defense_Data.xlsx
  Customer Region Customer Country Customer Operator Supplier Region Supplier Country Domestic Content   Market Segment                                                                                                                                                                                                   Market Segment Evidence                                          Market Segment Reason System Type (General)                                                                                                                                                                                            System Type (General) Evidence                                                  System Type (General) Reason System Type (Specific)                                                                                                                                                                                           System